In [1]:
function Lorenz(::Type{T}, t, u) where T<:AbstractFloat
    
    σ = 10 # parameters for Lorenz
    ρ = 28 
    β = 8/3
    y1::T = σ*(u[2]-u[1])
    y2::T = ρ*u[1]-u[2]-u[1]*u[3]
    y3::T = u[1]*u[2]-β*u[3]
   
    return [y1; y2; y3]
end

Lorenz (generic function with 1 method)

In [2]:
using LinearAlgebra
function FE(::Type{T}, f, a, b, u0, N) where T<:AbstractFloat
    ndf = length(u0)
    u = Matrix{T}(undef, ndf, N+1)
    dt = (b-a)/N
    t = Vector(range(a, b, length=N+1))
    u[:, 1] = u0
    for i in 1:N
        u[:, i+1] = u[:, i] .+ dt * Lorenz(T ,t[i], u[:, i])
    end
    return t, u
end

FE (generic function with 1 method)

In [3]:
a = 0
b = 1
u0 = [20; 5; -5]
N = 10
t, u = FE(Float64, Lorenz, a, b, u0, N)

([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], [20.0 5.0 … 2.2081236989294834e14 -3.8433116135573024e21; 5.0 70.5 … -3.8433116135573024e21 -1.849215884989871e35; -5.0 6.333333333333334 … 8.374602771965861e21 -8.486507456266178e34])

In [4]:
# Parareal method
function parareal(::Type{T}, a, b, u0, N, F, G, K) where T<:AbstractFloat
    # a= start time value (t0)
    # b= End time value (tend)
    # N= Number of Subinterval
    # u0=initial Solution
    # F= Fine Solver
    # G= Coarse Solver
    # K= Parareal iteration
    ndf=length(u0)
    tval=Vector(range(a, b, N+1))
    # Storing Fine Solution on each time step for comparison
    Ufine=Matrix{T}(undef, ndf, N+1)
    Ufine[:,1]=u0
    for i in 1:N
        Ufine[:, i+1]=F(T, tval[i], tval[i+1], Ufine[:, i])
    end
    # Initializing Uk
    Uk::Vector{Vector{T}}=[zeros(ndf, N+1) for k in 1:K+1]
    #Storing the initial solution
    for k in 1:K+1
        Uk[k][1]=u0
    end
    # Parareal Initialization
    for j in 1:N
        #Uk[1][j+1]=G(tval[j], tval[j+1], Uk[1][j])
        Uk[1][j+1]= 10.0 # May take random initial guess
    end
    # Parareal loop
    for k in 1:K
        for i in 1:N
            Fk0=F(T, tval[i], tval[i+1], Uk[k][i])
            Gk0=G(T, tval[i], tval[i+1], Uk[k][i])
            Gk1=G(T, tval[i], tval[i+1], Uk[k+1][i])
            
            Uk[k+1][i+1]=Fk0 + Gk1 - Gk0
        end
    end
    
    return Uk, Ufine, tval
  
end

parareal (generic function with 1 method)

In [5]:
function Fs(::Type{T},a, b, u0) where T<:AbstractFloat
    t, u=FE(T, Lorenz, a, b, u0, N)
    return u[end]
end

Fs (generic function with 1 method)

In [6]:
function Gs(::Type{T},a, b, u0) where T<:AbstractFloat
    t, u=FE(T, Lorenz, a, b, u0, N)
    return u[end]
end

Gs (generic function with 1 method)

In [7]:
# Convergence Analysis in uniform double precision
a=0
b=1
u0=[20; -5; 5]

#exact=exp.((t.^3)/3)


nF=1000 # Fine grid
nG=15 # Coarse grid
K=100 # Number of iteration
p = 3
N = zeros(p)
for i in 1:p
    N[i] = Int(floor(3^(i+2)))
end

err=zeros(p, K+1)
# Parareal Solution
for i in 1:p
    Uk, Ufine, tval=parareal(Float64, a, b, u0, convert(Int, N[i]), Fs, Gs, K)

    # stroing errors
    
    for k in 1:K+1
        err[i, k]=norm(Uk[k]-Ufine)
    end

end

using Plots
plot(err[1,:], yaxis=:log, xlabel="Number of iterations", ylabel="error", label="N=3^3")
for j in 2:p
    plot!(err[j,:], yaxis=:log, xlabel="Number of iterations", ylabel="error", label="N=3^$(j+2)")
end
display(plot!(legend=:topright))

LoadError: MethodError: no method matching +(::Vector{Float64}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:87
[0m  +([91m::UniformScaling[39m, ::Number) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:145
[0m  ...